# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLOptionsHW").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/07 18:19:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [2]:
path = '/Users/apple/Desktop/Learning_PySpark/Datasets/'
data = spark.read.csv(path+'googleplaystore.csv',inferSchema=True, header=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [3]:
data.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


As well as the schema to make sure all the column types were correctly infered

In [4]:
data.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [6]:
from pyspark.sql.types import IntegerType, FloatType
newdata = data.withColumn("Rating", data["Rating"].cast(FloatType())) \
            .withColumn("Reviews", data["Reviews"].cast(IntegerType())) \
            .withColumn("Price", data["Price"].cast(IntegerType()))
print(newdata.printSchema())
newdata.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [8]:
newdata.createOrReplaceTempView("df")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [13]:
spark.sql('select * from df where Rating>4.1').toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510.0,8.7M,"5,000,000+",Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
1,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644.0,25M,"50,000,000+",Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967.0,2.8M,"100,000+",Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
3,Paper flowers instructions,ART_AND_DESIGN,4.4,167.0,5.6M,"50,000+",Free,0.0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
4,Garden Coloring Book,ART_AND_DESIGN,4.4,13791.0,33M,"1,000,000+",Free,0.0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,Sya9a Maroc - FR,FAMILY,4.5,38.0,53M,"5,000+",Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
7561,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4.0,3.6M,100+,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
7562,Parkinson Exercices FR,MEDICAL,NaN,3.0,9.5M,"1,000+",Free,0.0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
7563,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114.0,Varies with device,"1,000+",Free,0.0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [23]:
result = spark.sql("select App, Rating from df where Category in ('COMICS') and Rating>4.5")
result.toPandas()

,App,Rating
0,Manga Master - Best manga & comic reader,4.6
1,GANMA! - All original stories free of charge f...,4.7
2,Röhrich Werner Soundboard,4.7
3,Unicorn Pokez - Color By Number,4.8
4,Manga - read Thai translation,4.6
5,Dragon Ball Wallpaper - Ringtones,4.7
6,Children's cartoons (Mithu-Mina-Raju),4.6
7,【Ranobbe complete free】 Novelba - Free app tha...,NaN
8,Faustop Sounds,4.7
9,Hojiboy Tojiboyev Life Hacks,5.0


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [25]:
spark.sql("select Category, sum(Reviews) as total_Reviews from df group by Category order by total_Reviews desc limit 1").show()

+--------+-------------+
|Category|total_Reviews|
+--------+-------------+
|    GAME|   1585422349|
+--------+-------------+



## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [28]:
spark.sql("select App, Reviews from df order by Reviews desc limit 1").show()

+--------+--------+
|     App| Reviews|
+--------+--------+
|Facebook|78158306|
+--------+--------+



## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [42]:
spark.sql("select App from df where App like '%dating%' ").show(truncate=False)

+--------------------------------------------------+
|App                                               |
+--------------------------------------------------+
|Meet, chat & date. Free dating app - Chocolate app|
|Friend Find: free chat + flirt dating app         |
|Spine- The dating app                             |
|Princess Closet : Otome games free dating sim     |
|happn – Local dating app                          |
+--------------------------------------------------+



## 6. Use SQL Transformer to display how many free apps there are in this list

In [46]:
from pyspark.ml.feature import SQLTransformer

sql_obj = SQLTransformer(statement="select count(*) as total_free_apps from __THIS__ where Type='Free';")
sql_obj.transform(newdata).show()

+---------------+
|total_free_apps|
+---------------+
|          10037|
+---------------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [47]:
sql_obj = SQLTransformer(statement="select Genres, count(*) as total_count from __THIS__ group by Genres order by total_count desc limit 1;")

sql_obj.transform(newdata).toPandas()

,Genres,total_count
0,Tools,842


## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [49]:
sql_obj = SQLTransformer(statement="select App from __THIS__ where Genres='Tools' and Reviews>100;")

sql_obj.transform(newdata).toPandas()

,App
0,Moto File Manager
1,Google
2,Google Translate
3,Moto Display
4,Motorola Alert
...,...
530,ChopAssistant
531,Reindeer VPN - Proxy VPN
532,Inf VPN - Global Proxy & Unlimited Free WIFI VPN
533,FR Roster


## That's all folks! Great job!